In [1]:
!pip install pandas scikit-learn

In [2]:
import pandas as pd

# Đọc dữ liệu từ tệp CSV
file_path = 'products.csv'
products_df = pd.read_csv(file_path)

# Xử lý giá sản phẩm để chuyển từ chuỗi sang số
products_df['price'] = products_df['price'].str.replace('.', '').astype(int)
products_df.head()

,category,sub_category,title,description,price,image_url
0,Điện thoại,Iphone,iPhone 15 Pro Max,Chip Apple A17 Pro 6 nhân | RAM: 8 GB | Dung l...,29590000,https://cdn.tgdd.vn/Products/Images/42/305658/...
1,Điện thoại,Iphone,iPhone 13,Chip Apple A15 Bionic | RAM: 4 GB | Dung lượng...,13690000,https://cdn.tgdd.vn/Products/Images/42/223602/...
2,Điện thoại,Iphone,iPhone 15 Pro,Chip Apple A17 Pro 6 nhân | RAM: 8 GB | Dung l...,24590000,https://cdn.tgdd.vn/Products/Images/42/299033/...
3,Điện thoại,Iphone,iPhone 15 Plus,Chip Apple A16 Bionic | RAM: 6 GB | Dung lượng...,22190000,https://cdn.tgdd.vn/Products/Images/42/303891/...
4,Điện thoại,Iphone,iPhone 15,Chip Apple A16 Bionic | RAM: 6 GB | Dung lượng...,18990000,https://cdn.tgdd.vn/Products/Images/42/281570/...


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Tạo ma trận đặc trưng từ mô tả sản phẩm
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(products_df['description'])

# Tính toán độ tương đồng cosine giữa các sản phẩm
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Hàm để gợi ý sản phẩm dựa trên nội dung
def get_content_based_recommendations(title, cosine_sim=cosine_sim):
    # Lấy chỉ số của sản phẩm có tiêu đề tương ứng
    idx = products_df[products_df['title'] == title].index[0]

    # Lấy danh sách các điểm tương đồng của sản phẩm này với các sản phẩm khác
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sắp xếp sản phẩm theo điểm tương đồng
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Lấy top 10 sản phẩm tương tự (trừ sản phẩm hiện tại)
    sim_scores = sim_scores[1:11]

    # Lấy chỉ số của các sản phẩm tương tự
    product_indices = [i[0] for i in sim_scores]

    # Trả về các sản phẩm tương tự
    return products_df.iloc[product_indices][['title', 'price', 'image_url']]

# Kiểm tra hàm với sản phẩm "iPhone 15 Pro Max"
get_content_based_recommendations('iPhone 15 Pro Max')

,title,price,image_url
2,iPhone 15 Pro,24590000,https://cdn.tgdd.vn/Products/Images/42/299033/...
7,iPhone 14 Pro Max,26990000,https://cdn.tgdd.vn/Products/Images/42/251192/...
3,iPhone 15 Plus,22190000,https://cdn.tgdd.vn/Products/Images/42/303891/...
4,iPhone 15,18990000,https://cdn.tgdd.vn/Products/Images/42/281570/...
15,Samsung Galaxy S24 Ultra 5G,26490000,https://cdn.tgdd.vn/Products/Images/42/307174/...
11,Samsung Galaxy A55 5G,11490000,https://cdn.tgdd.vn/Products/Images/42/322096/...
17,Samsung Galaxy S24+ 5G,20490000,https://cdn.tgdd.vn/Products/Images/42/307172/...
56,Xiaomi 13T Pro 5G,13790000,https://cdn.tgdd.vn/Products/Images/42/309816/...
16,Samsung Galaxy S24 5G,19790000,https://cdn.tgdd.vn/Products/Images/42/319665/...
10,Samsung Galaxy S23 Ultra 5G,21990000,https://cdn.tgdd.vn/Products/Images/42/249948/...


In [4]:
import numpy as np
import random

# Giả lập danh sách user_id và product_id
user_ids = np.arange(1, 101)  # 100 người dùng
product_ids = products_df.index.to_list()  # ID sản phẩm từ dữ liệu products_df

# Giả lập dữ liệu đánh giá
ratings_data = []

for user_id in user_ids:
    # Mỗi người dùng đánh giá từ 5 đến 15 sản phẩm
    num_ratings = random.randint(5, 15)
    rated_products = random.sample(product_ids, num_ratings)
    for product_id in rated_products:
        rating = random.randint(1, 5)  # Đánh giá từ 1 đến 5
        ratings_data.append((user_id, product_id, rating))

# Tạo DataFrame từ dữ liệu đánh giá
ratings_df = pd.DataFrame(ratings_data, columns=['user_id', 'product_id', 'rating'])
ratings_df.head()

,user_id,product_id,rating
0,1,51,2
1,1,58,4
2,1,140,2
3,1,183,5
4,1,275,1


In [5]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

# Chuẩn bị dữ liệu ma trận người dùng - sản phẩm
user_product_matrix = ratings_df.pivot(index='user_id', columns='product_id', values='rating').fillna(0)
user_product_sparse = csr_matrix(user_product_matrix)

# Huấn luyện mô hình SVD
svd = TruncatedSVD(n_components=50, random_state=42)
user_factors = svd.fit_transform(user_product_sparse)
product_factors = svd.components_.T

# Hàm để gợi ý sản phẩm cho người dùng dựa trên cộng tác
def get_collaborative_recommendations(user_id, num_recommendations=5):
    user_idx = user_id - 1  # Giả sử user_id là từ 1 đến 100
    user_ratings = user_factors[user_idx, :]
    scores = user_ratings.dot(product_factors.T)
    top_product_indices = np.argsort(scores)[::-1][:num_recommendations]
    return products_df.iloc[top_product_indices][['title', 'price', 'image_url']]

# Kiểm tra hàm với user_id = 1
get_collaborative_recommendations(1)

,title,price,image_url
179,Acer Aspire 3 A314 42P R3B3 R7 5700U (NX.KSFSV...,12290000,https://cdn.tgdd.vn/Products/Images/44/322222/...
198,Dell Inspiron 15 3520 i3 1215U (71003264),11790000,https://cdn.tgdd.vn/Products/Images/44/309293/...
285,Huawei Watch Fit 3 43.2mm dây silicone,2790000,https://cdn.tgdd.vn/Products/Images/7077/32537...
58,Xiaomi Redmi Note 13 Pro+ 5G,9990000,https://cdn.tgdd.vn/Products/Images/42/312965/...
196,Dell Inspiron 15 3530 i7 1355U (N3530I716W1),24490000,https://cdn.tgdd.vn/Products/Images/44/314841/...


In [6]:
def get_combined_recommendations(title, user_id, num_recommendations=5):
    # Gợi ý dựa trên nội dung
    content_recommendations = get_content_based_recommendations(title)
    
    # Gợi ý dựa trên cộng tác
    collab_recommendations = get_collaborative_recommendations(user_id)
    
    # Kết hợp và loại bỏ các sản phẩm trùng lặp
    combined_recommendations = pd.concat([content_recommendations, collab_recommendations]).drop_duplicates().head(num_recommendations)
    
    return combined_recommendations

# Gợi ý kết hợp cho "iPhone 15 Pro Max" và người dùng có user_id = 1
combined_recommendations = get_combined_recommendations('iPhone 15 Pro Max', 1)
combined_recommendations

,title,price,image_url
2,iPhone 15 Pro,24590000,https://cdn.tgdd.vn/Products/Images/42/299033/...
7,iPhone 14 Pro Max,26990000,https://cdn.tgdd.vn/Products/Images/42/251192/...
3,iPhone 15 Plus,22190000,https://cdn.tgdd.vn/Products/Images/42/303891/...
4,iPhone 15,18990000,https://cdn.tgdd.vn/Products/Images/42/281570/...
15,Samsung Galaxy S24 Ultra 5G,26490000,https://cdn.tgdd.vn/Products/Images/42/307174/...


In [7]:
# Gợi ý kết hợp cho "iPhone 13" và người dùng có user_id = 1
combined_recommendations_iphone13 = get_combined_recommendations('iPhone 13', 1)
print(combined_recommendations_iphone13)

# Gợi ý kết hợp cho "Samsung Galaxy S24 Ultra 5G" và người dùng có user_id = 1
combined_recommendations_galaxy_s24 = get_combined_recommendations('Samsung Galaxy S24 Ultra 5G', 1)
print(combined_recommendations_galaxy_s24)


               title     price  \
8     iPhone 14 Plus  19390000   
9          iPhone 14  16490000   
6          iPhone 12  11590000   
7  iPhone 14 Pro Max  26990000   
3     iPhone 15 Plus  22190000   

                                           image_url  
8  https://cdn.tgdd.vn/Products/Images/42/245545/...  
9  https://cdn.tgdd.vn/Products/Images/42/240259/...  
6  https://cdn.tgdd.vn/Products/Images/42/213031/...  
7  https://cdn.tgdd.vn/Products/Images/42/251192/...  
3  https://cdn.tgdd.vn/Products/Images/42/303891/...  
                          title     price  \
10  Samsung Galaxy S23 Ultra 5G  21990000   
20    Samsung Galaxy Z Fold5 5G  31990000   
17       Samsung Galaxy S24+ 5G  20490000   
11        Samsung Galaxy A55 5G  11490000   
16        Samsung Galaxy S24 5G  19790000   

                                            image_url  
10  https://cdn.tgdd.vn/Products/Images/42/249948/...  
20  https://cdn.tgdd.vn/Products/Images/42/301608/...  
17  https://cdn.tgdd.vn/Pr

In [8]:
import pickle

# Lưu mô hình SVD
with open('svd_model.pkl', 'wb') as f:
    pickle.dump(svd, f)

# Lưu ma trận đặc trưng TF-IDF và cosine similarity
with open('tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

with open('cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)

# Lưu dataframes
products_df.to_pickle('products_df.pkl')
ratings_df.to_pickle('ratings_df.pkl')